# HRRR Forecast Collection Best Time Series 
This notebook demonstrates how to access a collection of GRIB2 files as a single cloud-friendly dataset in xarray. 

We first use [kerchunk](https://github.com/fsspec/kerchunk) to create the JSON file that is then read by the zarr libaray using a referenceFileSystem created by fsspec.   


In [ ]:
import xarray as xr
import datetime as dt
import pandas as pd
import dask
import json
import fsspec
from kerchunk.combine import MultiZarrToZarr

## Examine resulting dataset

In [ ]:
fs = fsspec.filesystem(
    "reference", 
    fo='s3://esip-qhub-public/noaa/hrrr/hrrr_best.json', 
    ref_storage_args={'requester_pays': True, 'skip_instance_cache': True},
    remote_protocol='s3', 
    remote_options={'anon': True},
)
m = fs.get_mapper("")
ds2 = xr.open_dataset(
    m, 
    engine="zarr", 
    backend_kwargs={"consolidated": False}, 
    chunks={'valid_time':1},
)

In [ ]:
var='t2m'

In [ ]:
print(f'Size:{ds2[var].nbytes/1e9} GB')

In [ ]:
ds2[var]

Hvplot wants lon [-180,180], not [0,360]:

In [ ]:
ds2 = ds2.assign_coords(longitude=(((ds2.longitude + 180) % 360) - 180))

In [ ]:
now = dt.datetime.utcnow().strftime('%Y-%m-%d %H:00:00')
print(now)

With 30 worker cluster, takes 50 seconds to display, and 15 seconds to change the time step
after closing the dask client, it takes 30 seconds to display, 8 seconds to display a time step

In [ ]:
ds2[var].sel(valid_time=now).hvplot.quadmesh(x='longitude', y='latitude', geo=True,
                                                rasterize=True, cmap='turbo', title=now)

hvplot has a slider for time steps, but we want a dropdown list, so we use Panel.  Let's add a tile layer from Open Street Map also. 

In [ ]:
extra_dims = list(ds2[var].dims[:-2])
mesh = ds2[var].hvplot.quadmesh(x='longitude', y='latitude', rasterize=True, geo=True, tiles='OSM', cmap='turbo')
pn.panel(mesh, widgets={k: pn.widgets.Select for k in extra_dims})

#### Extract a time series at a point
We are reading GRIB2 files, which compress the entire spatial domain as a single chunk.  Therefore reading all the time values at a single point actually needs to load and uncompress *all* the data for that variable. 

In [ ]:
%%time
ds2[var][:,500,500].hvplot(x='valid_time', grid=True)

#### Compute the time-mean temperature over the entire dataset

In [ ]:
%%time
da = ds2[var].mean(dim='valid_time').compute()

In [ ]:
da.hvplot.quadmesh(x='longitude', y='latitude', rasterize=True, geo=True, tiles='OSM', cmap='turbo')

#### Close the client and shutdown the cluster